In [1]:
import pandas as pd
import numpy as np
import requests
import tensorflow as tf
import pyttsx3
import geocoder
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score

# Initialize text-to-speech engine
engine = pyttsx3.init()

def speak(text):
    print(text)  # Ensuring text is written
    engine.say(text)
    engine.runAndWait()

# Function to get weather data using WeatherAPI.com
def get_weather_data(api_key, location):
    url = f"https://api.weatherapi.com/v1/current.json?key={api_key}&q={location}&aqi=no"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "current" not in data:
            print(f"Invalid response for {location}: {data}")
            return None

        return {
            "Max_Temp": data["current"].get("temp_c", 0),
            "Rainfall": data["current"].get("precip_mm", 0),
            "Relative_Humidity": data["current"].get("humidity", 0),
            "Wind_Speed": data["current"].get("wind_kph", 0),
            "Cloud_Coverage": data["current"].get("cloud", 0)
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching weather data for {location}: {e}")
        return None

# Load dataset
df = pd.read_csv("c:/Users/harsh/Downloads/FloodPrediction.csv")
df['Flood?'] = df['Flood?'].fillna(0)

features = ["Max_Temp", "Rainfall", "Relative_Humidity", "Wind_Speed", "Cloud_Coverage"]
X = df[features]
y_flood = df["Flood?"]
label_encoder = LabelEncoder()
y_flood = label_encoder.fit_transform(y_flood)

X_train, X_test, y_train, y_test = train_test_split(X, y_flood, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

In [3]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)

Epoch 1/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9409 - loss: 0.1547 - val_accuracy: 0.9483 - val_loss: 0.1371
Epoch 2/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9435 - loss: 0.1511 - val_accuracy: 0.9471 - val_loss: 0.1378
Epoch 3/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9398 - loss: 0.1524 - val_accuracy: 0.9477 - val_loss: 0.1383
Epoch 4/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9425 - loss: 0.1521 - val_accuracy: 0.9486 - val_loss: 0.1373
Epoch 5/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9453 - loss: 0.1463 - val_accuracy: 0.9462 - val_loss: 0.1376
Epoch 6/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9439 - loss: 0.1468 - val_accuracy: 0.9468 - val_loss: 0.1370
Epoch 7/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9421 - loss: 0.1453 - val_accuracy: 0.9477 - val_loss: 0.1369
Epoch 8/50
822/822 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9420 - loss: 0.1466 - val_accuracy: 0.

In [4]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
speak(f"Model Accuracy: {accuracy * 100:.2f} percent")
model.save(r"C:\Users\harsh\OneDrive\Desktop\flood_prediction\flood_prediction_model.h5")

def predict_flood(api_key):
    cities = ["Mumbai", "Delhi", "Kolkata", "Mysuru", "Bangalore"]
    results = {}

    for city in cities:
        weather_data = get_weather_data(api_key, city)
        if weather_data:
            weather_df = pd.DataFrame([weather_data])
            weather_scaled = scaler.transform(weather_df)

            prediction = model.predict(weather_scaled)[0][0]

            if prediction > 0.7:
                flood_risk = "High Risk Flood Area 🚨"
            elif prediction > 0.4:
                flood_risk = "Moderate Risk Area ⚠️"
            else:
                flood_risk = "Flood Free Area ✅"

            results[city] = flood_risk
            speak(f"Flood Prediction for {city}: {flood_risk}")
        else:
            results[city] = "No data available"
            print(f"No data available for {city}")
    
    return results


129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Model Accuracy: 94.13 percent


In [5]:
api_key = "940f21d73c2f4d44b88120335251402 "
predict_flood(api_key)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Flood Prediction for Mumbai: Flood Free Area ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Flood Prediction for Delhi: Flood Free Area ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Flood Prediction for Kolkata: Flood Free Area ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Flood Prediction for Mysuru: Flood Free Area ✅
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Flood Prediction for Bangalore: Flood Free Area ✅


{'Mumbai': 'Flood Free Area ✅',
 'Delhi': 'Flood Free Area ✅',
 'Kolkata': 'Flood Free Area ✅',
 'Mysuru': 'Flood Free Area ✅',
 'Bangalore': 'Flood Free Area ✅'}